In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
#!/usr/bin/env python3
"""
Script para descargar todos los archivos .txt.gz de los MITgcm-support Archives,
descomprimirlos y agrupar su contenido en un único archivo de texto.

Requisitos:
    pip install requests beautifulsoup4
"""

import os
import re
import gzip
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "http://mailman.mitgcm.org/pipermail/mitgcm-support/"

def get_txtgz_links():
    """Extrae los enlaces a archivos .txt.gz desde la página principal."""
    response = requests.get(BASE_URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    txtgz_links = []
    # Busca todos los <a> cuyo href termine en .txt.gz
    for a in soup.find_all('a', href=True):
        href = a['href']
        if href.endswith(".txt.gz"):
            full_url = urljoin(BASE_URL, href)
            txtgz_links.append(full_url)
    return txtgz_links

def download_and_decompress(url):
    """Descarga un archivo .txt.gz de la URL y lo descomprime.
    Retorna el contenido en texto (utf-8).
    """
    print(f"Descargando {url} ...")
    response = requests.get(url)
    response.raise_for_status()
    # Descomprimir usando gzip; response.content es de tipo bytes
    try:
        decompressed_bytes = gzip.decompress(response.content)
    except Exception as e:
        print(f"Error al descomprimir {url}: {e}")
        return ""
    # Decodifica (la codificación puede variar; usamos utf-8 por defecto)
    try:
        text = decompressed_bytes.decode('utf-8')
    except UnicodeDecodeError:
        # Si falla, intenta con latin1
        text = decompressed_bytes.decode('latin1')
    return text

def main():
    # Archivo de salida donde se agrupará todo el contenido
    output_filename = "todos_archivos.txt"
    
    links = get_txtgz_links()
    if not links:
        print("No se encontraron archivos .txt.gz en la página.")
        return
    print(f"Se encontraron {len(links)} archivos .txt.gz.")

    # Abre el archivo de salida en modo escritura (sobrescribe si existe)
    with open(output_filename, "w", encoding="utf-8") as outfile:
        for url in links:
            # Extrae el nombre base para usarlo como separador en el output
            file_basename = os.path.basename(url)
            header = f"\n{'='*40}\nArchivo: {file_basename}\n{'='*40}\n"
            outfile.write(header)
            content = download_and_decompress(url)
            outfile.write(content)
            print(f"Procesado: {file_basename}")

    print(f"\nTodos los archivos se han agrupado en: {output_filename}")

if __name__ == "__main__":
    main()


Se encontraron 261 archivos .txt.gz.
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2025-February.txt.gz ...
Procesado: 2025-February.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2025-January.txt.gz ...
Procesado: 2025-January.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-December.txt.gz ...
Procesado: 2024-December.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-November.txt.gz ...
Procesado: 2024-November.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-October.txt.gz ...
Procesado: 2024-October.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-September.txt.gz ...
Procesado: 2024-September.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-August.txt.gz ...
Procesado: 2024-August.txt.gz
Descargando http://mailman.mitgcm.org/pipermail/mitgcm-support/2024-July.txt.gz ...
Procesado: 2024-July.txt.gz
Descargando http: